**Objective**
<br>Parse job bulletin text files and create output dataframe with the structure mentioned in "Sample job class export template.csv"

<br>**Columns Added** 
>        'FILE_NAME', 'JOB_CLASS_TITLE', 'JOB_CLASS_NO', 'REQUIREMENT_SET_ID', 
       'REQUIREMENT_SUBSET_ID', 'ENTRY_SALARY_GEN', 'ENTRY_SALARY_DWP', 'OPEN_DATE',
       'SCHOOL_TYPE','EDUCATION_MAJOR', 'JOB_DUTIES_1', 'JOB_DUTIES_2', 'EXAM_TYPE'

<br><br>**Columns Left**
>      'EDUCATION_YEARS', 'EXPERIENCE_LENGTH', 'FULL_TIME_PART_TIME',
       'EXP_JOB_CLASS_TITLE', 'EXP_JOB_CLASS_ALT_RESP',
       'EXP_JOB_CLASS_FUNCTION', 'COURSE_COUNT', 'COURSE_LENGTH',
       'COURSE_SUBJECT', 'MISC_COURSE_DETAILS', 'DRIVERS_LICENSE_REQ',
       'DRIV_LIC_TYPE', 'ADDTL_LIC','Benefits'

In [1]:
import os
import pandas as pd,numpy as np
import re
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from collections import Counter
import pprint
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [2]:
bulletin_dir = '../input/cityofla/CityofLA/Job Bulletins/'
additional_data_dir = '../input/cityofla/CityofLA/Additional data/'

Assumption after looking at the data in text files: Headings are written in upper case letters.
<br>I've used this assumption to parse the text

In [3]:
headings = {}
m=0
for filename in os.listdir(bulletin_dir):
    with open(bulletin_dir + "/" + filename, 'r', errors='ignore') as f:
        for line in f.readlines():
            line = line.replace("\n","").replace("\t","").replace(":","").strip()
#             m+=1
#             if m==8:
#                 break
            if line.isupper():
                if line not in headings.keys():
                    headings[line] = 1
                else:
                    count = int(headings[line])
                    headings[line] = count+1
#     break

In [4]:
del headings['$103,606 TO $151,484'] #This is not a heading, it's an Annual Salary component
headingsFrame = []
for i,j in (sorted(headings.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)):
    headingsFrame.append([i,j])
headingsFrame = pd.DataFrame(headingsFrame)
headingsFrame.columns = ["Heading","Count"]
#headingsFrame.head()

In [5]:
#Add 'FILE_NAME', 'JOB_CLASS_TITLE', 'JOB_CLASS_NO' ,'OPEN_DATE'
data_list = []
for filename in os.listdir(bulletin_dir):
    with open(bulletin_dir + "/" + filename, 'r', errors='ignore') as f:
        job_class_title = ''
        for line in f.readlines():
            #Insert code to parse job bulletins
            if "Open Date:" in line:
                job_bulletin_date = line.split("Open Date:")[1].split("(")[0].strip()
            if "Class Code:" in line:
                job_class_no = line.split("Class Code:")[1].strip()
            if len(job_class_title)<2 and len(line.strip())>1:
                job_class_title = line.strip()
        data_list.append([filename, job_bulletin_date, job_class_title, job_class_no])

In [6]:
df = pd.DataFrame(data_list)
df.columns = ["FILE_NAME", "OPEN_DATE", "JOB_CLASS_TITLE", "JOB_CLASS_NO"]
df.head()

,FILE_NAME,OPEN_DATE,JOB_CLASS_TITLE,JOB_CLASS_NO
0,WATER SERVICE REPRESENTATIVE 1693 111717.txt,11-17-17,WATER SERVICE REPRESENTATIVE,1693
1,HARBOR PLANNING AND ECONOMIC ANALYST 9224 1118...,11-18-16,HARBOR PLANNING AND ECONOMIC ANALYST,9224
2,MANAGING WATER UTILITY ENGINEER 9406 032417 RE...,03-24-17,MANAGING WATER UTILITY ENGINEER,9406
3,VETERINARY TECHNICIAN 2369 020599 REV 120417.txt,02-05-99,VETERINARY TECHNICIAN,2369
4,SENIOR UTILITY SERVICES SPECIALIST 3573 113018...,11-30-18,SENIOR UTILITY SERVICES SPECIALIST,3753


In [7]:
#Add 'REQUIREMENT_SET_ID','REQUIREMENT_SUBSET_ID'
requirements = []
requirementHeadings = [k for k in headingsFrame['Heading'].values if 'requirement' in k.lower()]
for filename in os.listdir(bulletin_dir):
    with open(bulletin_dir + "/" + filename, 'r', errors='ignore') as f:
        readNext = 0
        isNumber=0
        prevNumber=0
        prevLine=''
        
        for line in f.readlines():
            clean_line = line.replace("\n","").replace("\t","").replace(":","").strip()   
            if readNext == 0:                         
                if clean_line in requirementHeadings:
                    readNext = 1
            elif readNext == 1:
                if clean_line in headingsFrame['Heading'].values:
                    if isNumber>0:
                        requirements.append([filename,prevNumber,'',prevLine])
                    break
                elif len(clean_line)<2:
                    continue
                else:
                    rqrmntText = clean_line.split('.')
                    if len(rqrmntText)<2:
                        requirements.append([filename,'','',clean_line])
                    else:                        
                        if rqrmntText[0].isdigit():
                            if isNumber>0:
                                requirements.append([filename,prevNumber,'',prevLine])
                            isNumber=1
                            prevNumber=rqrmntText[0]
                            prevLine=clean_line
                        elif re.match('^[a-z]$',rqrmntText[0]):
                            requirements.append([filename,prevNumber,rqrmntText[0],prevLine+'-'+clean_line])
                            isNumber=0
                        else:
                            requirements.append([filename,'','',clean_line])

In [8]:
df_requirements = pd.DataFrame(requirements)
df_requirements.columns = ['FILE_NAME','REQUIREMENT_SET_ID','REQUIREMENT_SUBSET_ID','REQUIREMENT_TEXT']
df_requirements.head()

,FILE_NAME,REQUIREMENT_SET_ID,REQUIREMENT_SUBSET_ID,REQUIREMENT_TEXT
0,WATER SERVICE REPRESENTATIVE 1693 111717.txt,,,Three years of full-time paid experience in a ...
1,WATER SERVICE REPRESENTATIVE 1693 111717.txt,,,Education from an accredited college or univer...
2,HARBOR PLANNING AND ECONOMIC ANALYST 9224 1118...,1,,1. Graduation from an accredited four-year col...
3,HARBOR PLANNING AND ECONOMIC ANALYST 9224 1118...,2,,2. Two years of full-time paid professional ex...
4,MANAGING WATER UTILITY ENGINEER 9406 032417 RE...,1,,1. Two years of full-time paid experience at t...


In [9]:
#Check for one sample file 
df_requirements.loc[df_requirements['FILE_NAME']=='SYSTEMS ANALYST 1596 102717.txt']

,FILE_NAME,REQUIREMENT_SET_ID,REQUIREMENT_SUBSET_ID,REQUIREMENT_TEXT
1117,SYSTEMS ANALYST 1596 102717.txt,1,,1. Graduation from an accredited four-year col...
1118,SYSTEMS ANALYST 1596 102717.txt,2,a,2. Graduation from an accredited four-year col...
1119,SYSTEMS ANALYST 1596 102717.txt,2,b,2. Graduation from an accredited four-year col...
1120,SYSTEMS ANALYST 1596 102717.txt,2,c,2. Graduation from an accredited four-year col...
1121,SYSTEMS ANALYST 1596 102717.txt,3,a,3. Two years of full-time paid experience as a...
1122,SYSTEMS ANALYST 1596 102717.txt,3,b,3. Two years of full-time paid experience as a...


In [10]:
#Check for salary components
salHeadings = [k for k in headingsFrame['Heading'].values if 'salary' in k.lower()]
sal_list = []
for filename in os.listdir(bulletin_dir):
    with open(bulletin_dir + "/" + filename, 'r', errors='ignore') as f:
        readNext = 0
        for line in f.readlines():
            clean_line = line.replace("\n","").replace("\t","").replace(":","").strip()  
            if clean_line in salHeadings:
                readNext = 1
            elif readNext == 1:
                if clean_line in headingsFrame['Heading'].values:
                    break
                elif len(clean_line)<2:
                    continue
                else:
                    sal_list.append([filename, clean_line])


In [11]:
df_salary = pd.DataFrame(sal_list)
df_salary.columns = ['FILE_NAME','SALARY_TEXT']
df_salary['SALARY_TEXT'][6]

'$117,199 to $145,596'

In [12]:
files = []
for filename in os.listdir(bulletin_dir):
    files.append(filename)
    

# ENTRY_SALARY_GEN ENTRY_SALARY_DWP

In [13]:
#Add 'ENTRY_SALARY_GEN','ENTRY_SALARY_DWP'
pattern = r'\$?([0-9]{1,3},([0-9]{3},)*[0-9]{3}|[0-9]+)(.[0-9][0-9])?'
dwp_salary_list = {}
gen_salary_list = {}
for filename in files:
    for sal_text in df_salary.loc[df_salary['FILE_NAME']==filename]['SALARY_TEXT']:
        if 'department of water' in sal_text.lower():
            if filename in dwp_salary_list.keys():
                continue
            matches = re.findall(pattern+' to '+pattern, sal_text) 
            if len(matches)>0:
                salary_dwp = ' - '.join([x for x in matches[0] if x and not x.endswith(',')])
            else:
                matches = re.findall(pattern, sal_text)
                if len(matches)>0:
                    salary_dwp = matches[0][0]
                else:
                    salary_dwp = ''
            dwp_salary_list[filename]= salary_dwp
        else:
            if filename in gen_salary_list.keys():
                continue
            matches = re.findall(pattern+' to '+pattern, sal_text)
            if len(matches)>0:
                salary_gen = ' - '.join([x for x in matches[0] if x and not x.endswith(',')])
            else:
                matches = re.findall(pattern, sal_text)
                if len(matches)>0:
                    salary_gen = matches[0][0]
                else:
                    salary_gen = ''
            if len(salary_gen)>1:
                gen_salary_list[filename]= salary_gen

In [14]:
df_salary_dwp = pd.DataFrame(list(dwp_salary_list.items()), columns=['FILE_NAME','ENTRY_SALARY_DWP'])
df_salary_gen = pd.DataFrame(list(gen_salary_list.items()), columns=['FILE_NAME','ENTRY_SALARY_GEN'])

In [15]:
result = pd.merge(df, df_requirements, how='inner', left_on='FILE_NAME', right_on='FILE_NAME', sort=True)

result = pd.merge(result, df_salary_dwp, how='left', left_on='FILE_NAME', right_on='FILE_NAME', sort=True)

result = pd.merge(result, df_salary_gen, how='left', left_on='FILE_NAME', right_on='FILE_NAME', sort=True)

# ADD JOB_DUTIES_1 AND JOB_DUTIES_2

In [16]:
#ADD JOB_DUTIES_1 AND JOB_DUTIES_2
duties = {}
name2=[]
d1=[]
d2=[]
m=0
for filename in os.listdir(bulletin_dir):
    with open(bulletin_dir + "/" + filename, 'r', errors='ignore') as f:
        for line in f.readlines():
            line = line.replace("\n","").replace("\t","").replace(":","").strip()
            if 'DUTIES' in line:
                m=1
    #             print(line)
            if line.isupper() and line!='DUTIES':
                m=0
    #             print(line)
            if m==1 and line!='DUTIES' and line!='':
                
                if filename not in duties:
                    duties[filename]=line
                else:
                    d3=[duties[filename],line]
                    duties[filename]=d3
                    
for i in duties:
    name2.append(i)
    if len(duties[i])==2:
        d1.append(duties[i][0])
        d2.append(duties[i][1])
    else:
        d1.append(duties[i])
        d2.append(None)

df_duties = pd.DataFrame({'FILE_NAME': name2,'JOB_DUTIES_1': d1,'JOB_DUTIES_2': d2})
result = pd.merge(result, df_duties, how='left', left_on='FILE_NAME', right_on='FILE_NAME', sort=True)


#WITHOUT DUTIES

# APPARATUS OPERATOR 2121 071417 (1).txt
# ENGINEER OF FIRE DEPARTMENT 2131 111116.txt
# FIRE ASSISTANT CHIEF 2166 011218.txt
# FIRE BATTALION CHIEF 2152 030918.txt
# FIRE HELICOPTER PILOT 3563 081415 REV. 081815.txt
# FIRE INSPECTOR 2128 031717.txt
# Vocational Worker  DEPARTMENT OF PUBLIC WORKS.txt


# ADD the EXAM_type

In [17]:
### ADD the EXAM_type
exam_type = []
name1=[]
m=0
EXAM_TYPE=[]
for filename in os.listdir(bulletin_dir):
    with open(bulletin_dir + "/" + filename, 'r', errors='ignore') as f:
        for line in f.readlines():
            line = line.replace("\n","").replace("\t","").replace(":","").strip()
            if 'THIS EXAM' in line:
                m=1
    #             print(line)
            if 'discriminate' in line:
                m=0
    #             print(line)
            if m==1 and line!='' and not('THIS' in line):
                name1.append(filename)
                exam_type.append(line)
#                 print(filename,line)

for i in exam_type:
    if ('OPEN' in i) and ('INTER' in i):
        EXAM_TYPE.append('OPEN_INT_PROM')
    elif ('OPEN' in i) and ('PROMO' in i):
        EXAM_TYPE.append('OPEN_DEPT_PROM')
    elif 'OPEN' in i:
        EXAM_TYPE.append('OPEN')
    elif 'INTER' in i:
        EXAM_TYPE.append('INT_DEPT_PROM')
    else:
        EXAM_TYPE.append('DEPT_PROM')
            
df_exam_type = pd.DataFrame({'FILE_NAME': name1,'EXAM_TYPE': EXAM_TYPE})
result = pd.merge(result, df_exam_type, how='left', left_on='FILE_NAME', right_on='FILE_NAME', sort=True)


#WITHOUT EXAMINATION

# PILE DRIVER WORKER 3553 041417.txt
# Vocational Worker  DEPARTMENT OF PUBLIC WORKS.txt

In [18]:
#result.drop(columns=['REQUIREMENT_TEXT'], inplace=True)
result
# result.to_csv('result_table.csv',index=False)

,FILE_NAME,OPEN_DATE,JOB_CLASS_TITLE,JOB_CLASS_NO,REQUIREMENT_SET_ID,REQUIREMENT_SUBSET_ID,REQUIREMENT_TEXT,ENTRY_SALARY_DWP,ENTRY_SALARY_GEN,JOB_DUTIES_1,JOB_DUTIES_2,EXAM_TYPE
0,311 DIRECTOR 9206 041814.txt,04-18-14,311 DIRECTOR,9206,1,,1. One year of full-time paid experience as a ...,NaN,"125,175 - 155,514",A 311 Director is responsible for the successf...,None,OPEN_INT_PROM
1,311 DIRECTOR 9206 041814.txt,04-18-14,311 DIRECTOR,9206,2,,2. A Bachelor's degree from a recognized colle...,NaN,"125,175 - 155,514",A 311 Director is responsible for the successf...,None,OPEN_INT_PROM
2,311 DIRECTOR 9206 041814.txt,04-18-14,311 DIRECTOR,9206,3,,3. Eight years of full-time paid experience in...,NaN,"125,175 - 155,514",A 311 Director is responsible for the successf...,None,OPEN_INT_PROM
3,ACCOUNTANT 1513 062218.txt,06-22-18,ACCOUNTANT,1513,,,Graduation from an accredited four-year colleg...,NaN,"49,903 - 72,996",An Accountant does professional accounting wor...,None,OPEN
4,ACCOUNTING CLERK 1223 071318.txt,07-13-18,ACCOUNTING CLERK,1223,,,Two years of full-time paid office clerical ex...,NaN,"49,005 - 71,618",An Accounting Clerk performs difficult and res...,None,OPEN_INT_PROM
5,ACCOUNTING CLERK 1223 071318.txt,07-13-18,ACCOUNTING CLERK,1223,,,Successful completion of 6 semester or 9 quart...,NaN,"49,005 - 71,618",An Accounting Clerk performs difficult and res...,None,OPEN_INT_PROM
6,ACCOUNTING RECORDS SUPERVISOR 1119 072718.txt,07-27-18,ACCOUNTING RECORDS SUPERVISOR,1119,,,Two years of full-time paid experience as an A...,NaN,"55,332 - 80,930","An Accounting Records Supervisor assigns, revi...",None,INT_DEPT_PROM
7,ADMINISTRATIVE ANALYST 1590 060118.txt,06-01-18,ADMINISTRATIVE ANALYST,1590,1,,1. One year of full-time paid professional exp...,NaN,"60,489 - 88,468",An Administrative Analyst performs professiona...,None,INT_DEPT_PROM
8,ADMINISTRATIVE ANALYST 1590 060118.txt,06-01-18,ADMINISTRATIVE ANALYST,1590,2,,2. Two years of full-time paid experience as a...,NaN,"60,489 - 88,468",An Administrative Analyst performs professiona...,None,INT_DEPT_PROM
9,ADMINISTRATIVE ANALYST 1590 060118.txt,06-01-18,ADMINISTRATIVE ANALYST,1590,,,A bachelor's degree from an accredited four-ye...,NaN,"60,489 - 88,468",An Administrative Analyst performs professiona...,None,INT_DEPT_PROM


# School Type

In [19]:
def getSchoolType(txt):
    txt1=''
    line = txt.lower().find('college or university')
    if line==-1: 
#         return ''
        txt1+=''
    else:
        txt1+= 'college or university '
    line1 = txt.lower().find('high school')
    if line1==-1:
        txt1+= ''
    else:
        txt1+='HIGH SCHOOL'
    
    line2 = txt.lower().find('apprenticeship')
    if line2==-1:
        txt1+= ''
    else:
        txt1+='apprenticeship'

    return txt1

result['SCHOOL_TYPE'] = result['REQUIREMENT_TEXT'].apply(lambda x: getSchoolType(x))
result

,FILE_NAME,OPEN_DATE,JOB_CLASS_TITLE,JOB_CLASS_NO,REQUIREMENT_SET_ID,REQUIREMENT_SUBSET_ID,REQUIREMENT_TEXT,ENTRY_SALARY_DWP,ENTRY_SALARY_GEN,JOB_DUTIES_1,JOB_DUTIES_2,EXAM_TYPE,SCHOOL_TYPE
0,311 DIRECTOR 9206 041814.txt,04-18-14,311 DIRECTOR,9206,1,,1. One year of full-time paid experience as a ...,NaN,"125,175 - 155,514",A 311 Director is responsible for the successf...,None,OPEN_INT_PROM,
1,311 DIRECTOR 9206 041814.txt,04-18-14,311 DIRECTOR,9206,2,,2. A Bachelor's degree from a recognized colle...,NaN,"125,175 - 155,514",A 311 Director is responsible for the successf...,None,OPEN_INT_PROM,college or university
2,311 DIRECTOR 9206 041814.txt,04-18-14,311 DIRECTOR,9206,3,,3. Eight years of full-time paid experience in...,NaN,"125,175 - 155,514",A 311 Director is responsible for the successf...,None,OPEN_INT_PROM,
3,ACCOUNTANT 1513 062218.txt,06-22-18,ACCOUNTANT,1513,,,Graduation from an accredited four-year colleg...,NaN,"49,903 - 72,996",An Accountant does professional accounting wor...,None,OPEN,college or university
4,ACCOUNTING CLERK 1223 071318.txt,07-13-18,ACCOUNTING CLERK,1223,,,Two years of full-time paid office clerical ex...,NaN,"49,005 - 71,618",An Accounting Clerk performs difficult and res...,None,OPEN_INT_PROM,
5,ACCOUNTING CLERK 1223 071318.txt,07-13-18,ACCOUNTING CLERK,1223,,,Successful completion of 6 semester or 9 quart...,NaN,"49,005 - 71,618",An Accounting Clerk performs difficult and res...,None,OPEN_INT_PROM,college or university
6,ACCOUNTING RECORDS SUPERVISOR 1119 072718.txt,07-27-18,ACCOUNTING RECORDS SUPERVISOR,1119,,,Two years of full-time paid experience as an A...,NaN,"55,332 - 80,930","An Accounting Records Supervisor assigns, revi...",None,INT_DEPT_PROM,
7,ADMINISTRATIVE ANALYST 1590 060118.txt,06-01-18,ADMINISTRATIVE ANALYST,1590,1,,1. One year of full-time paid professional exp...,NaN,"60,489 - 88,468",An Administrative Analyst performs professiona...,None,INT_DEPT_PROM,
8,ADMINISTRATIVE ANALYST 1590 060118.txt,06-01-18,ADMINISTRATIVE ANALYST,1590,2,,2. Two years of full-time paid experience as a...,NaN,"60,489 - 88,468",An Administrative Analyst performs professiona...,None,INT_DEPT_PROM,
9,ADMINISTRATIVE ANALYST 1590 060118.txt,06-01-18,ADMINISTRATIVE ANALYST,1590,,,A bachelor's degree from an accredited four-ye...,NaN,"60,489 - 88,468",An Administrative Analyst performs professiona...,None,INT_DEPT_PROM,college or university


In [20]:
df1=result[result['SCHOOL_TYPE']=='apprenticeship'].drop(columns=['OPEN_DATE', 'JOB_CLASS_TITLE', 'JOB_CLASS_NO','ENTRY_SALARY_DWP', 'ENTRY_SALARY_GEN', 'JOB_DUTIES_1', 'JOB_DUTIES_2','EXAM_TYPE','SCHOOL_TYPE'])
df1

,FILE_NAME,REQUIREMENT_SET_ID,REQUIREMENT_SUBSET_ID,REQUIREMENT_TEXT
19,AIR CONDITIONING MECHANIC 3774 041417.txt,1,,1. Completion of a recognized apprenticeship a...
125,ASBESTOS WORKER 3435 100518.txt,1,,1. Completion of a recognized apprenticeship a...
174,AUTO BODY BUILDER AND REPAIRER 3704 101317 rev...,1,,1. Completion of a recognized four-year appren...
178,AUTO ELECTRICIAN 3707 052215.txt,1,,1. Completion of a recognized apprenticeship a...
181,AUTO PAINTER 3721 022417.txt,1,,1. Completion of a recognized apprenticeship a...
196,BLACKSMITH 3733 082517 REV 090717.txt,1,,1. Completion of an apprenticeship as a blacks...
199,BOILERMAKER 3735 110714.txt,1,,1. Completion of a recognized apprenticeship ...
209,BUILDING INSPECTOR 4211 101918.txt,3,,3. Two years of full-time paid experience in c...
218,BUILDING MECHANICAL INSPECTOR 4251 082815 REV ...,3,,3. Two years of full-time paid experience in c...
232,CARPENTER 3344 011218.txt,1,,1. Completion of an apprenticeship as a carpen...


In [21]:
for i in df1['REQUIREMENT_TEXT']:
    print(i)

1. Completion of a recognized apprenticeship as a refrigeration and air conditioning fitter; or
1. Completion of a recognized apprenticeship as an Asbestos Worker; or
1. Completion of a recognized four-year apprenticeship program as an auto body and fender repairer or auto body builder; or
1. Completion of a recognized apprenticeship as an auto electrician; or
1. Completion of a recognized apprenticeship as an auto painter; or
1. Completion of an apprenticeship as a blacksmith; or
1.  Completion of a recognized apprenticeship as a boilermaker; or
3. Two years of full-time paid experience in carpentry, masonry, concrete, structural steel or framing and rough carpentry work following the completion of a recognized apprenticeship program; or
3. Two years of full-time paid experience in carpentry, masonry, electrical, plumbing, or mechanical work, in general residential construction following the completion of a recognized apprenticeship program; or
1. Completion of an apprenticeship as a 

In [22]:
displacy.render(doc[175],style='dep')

NameError: name 'doc' is not defined

#  Education major

In [23]:
import string
from spacy import displacy
from spacy.matcher import Matcher

tagger=nltk.pos_tag
tokenizer = word_tokenize
stop = stopwords.words('english')


df1['REQUIREMENT_TEXT']= df1['REQUIREMENT_TEXT'].apply(lambda x: x.lower())
# df1['REQUIREMENT_TEXT']= df1['REQUIREMENT_TEXT'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df1['REQUIREMENT_TEXT']= df1['REQUIREMENT_TEXT'].apply(lambda x: x.translate(str.maketrans('', '', string.digits)))



# ####################################################################################################################
# text= result['REQUIREMENT_TEXT'].apply(tokenizer)
# text= text.apply(tagger)
#text=text.apply(lambda x: [item for item in x if item not in stop])
doc= df1['REQUIREMENT_TEXT'].apply(nlp)

# displacy.render(doc[19],style='dep')

matcher = Matcher(nlp.vocab)
pattern = [{'POS': 'NOUN'},
           {'POS': 'ADP'},
           {'POS': 'DET'},
           {'POS': 'NOUN'},
           {'POS': 'CCONJ', 'OP': '?'},
           {'POS': 'NOUN', 'OP': '?'},
           {'POS': 'NOUN', 'OP': '?'},
           {'POS': 'Noun', 'OP': '?'}]


# text= result['REQUIREMENT_TEXT'].apply(tokenizer)
# # text= text.apply(lambda x: [item for item in x if item not in stop])
# # text = [word for word in text if word.isalpha()]
# # text

                                                    
# # def preprocess(txt):
# #     txt = nltk.word_tokenize(txt)
# #     txt = nltk.pos_tag(txt)
# #     return txt

In [24]:
matcher.add("apprenticeship",None, pattern)
matches = matcher(doc[219])
matches
for match_id, start, end in matches:
# Get the matched span by slicing the Doc
    span = doc[219][start:end]
    print(span.text)

KeyError: 219

In [25]:
matcher.add("apprenticeship",None, pattern)
# matches =[]
for i in doc:
    matches =[]
    matches.append(matcher(i))
    if matches[0]:
        print(i[matches[0][-1][1]:matches[0][-1][2]])
# matches
# for match_id, start, end in matches:
# # Get the matched span by slicing the Doc
#     span = doc[19][start:end]
#     print(span.text)

apprenticeship as a refrigeration and air conditioning
apprenticeship as an asbestos worker
program as an auto body
apprenticeship as an auto electrician
apprenticeship as an auto painter
apprenticeship as a blacksmith
apprenticeship as a boilermaker
apprenticeship as a carpenter or cabinetmaker
apprenticeship as a carpet layer
apprenticeship as a cement
completion of an apprenticeship program
completion of a state
apprenticeship as a locksmith
apprenticeship as a machinist
apprenticeship as a bricklayer
apprenticeship as a painter
certificate at the time
apprenticeship as a pipefitter
apprenticeship as a plumber
experience with the port
apprenticeship as a roofer
apprenticeship as a sheet metal worker
apprenticeship as a ship carpenter
apprenticeship as a sign painter
apprenticeship as an electrician
apprenticeship as a tile setter
apprenticeship as a tire repairer
submission of an apprenticeship certificate
apprenticeship as a welder


In [26]:
matches

[[(6237887334661923750, 5, 9)]]

In [27]:
displacy.render(doc[19],style='dep')

In [28]:
doc[0],doc[1],doc[2]

KeyError: 0

In [29]:
text= result['REQUIREMENT_TEXT'].apply(tokenizer)
text= text.apply(lambda x: [item for item in x if item not in stop])

In [30]:
for i in text:
    for t in i:
        if len(t)==1:
            i.remove(t)
text

0       [., One, year, full-time, paid, experience, Se...
1       [., Bachelor, 's, degree, recognized, college,...
2       [Eight, years, full-time, paid, experience, ca...
3       [Graduation, accredited, four-year, college, u...
4       [Two, years, full-time, paid, office, clerical...
5       [Successful, completion, semester, quarter, un...
6       [Two, years, full-time, paid, experience, Acco...
7       [., One, year, full-time, paid, professional, ...
8       [., Two, years, full-time, paid, experience, M...
9       [bachelor, 's, degree, accredited, four-year, ...
10      [One, year, full-time, paid, experience, City,...
11      [Six, months, full-time, paid, experience, cle...
12      [., Two, years, full-time, paid, experience, C...
13      [., Two, years, full-time, paid, experience, C...
14      [., Two, years, full-time, paid, experience, C...
15      [520, hours, paid, experience, City, Los, Ange...
16      [Must, licensed, a-a, ., Registered, Nurse, ce...
17      [Must,